# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [16]:
# Calculations and counts
grouped_by_school = school_data_complete.groupby('school_name')
num_schools = len(grouped_by_school)
num_students = len(school_data_complete)
tot_budget = sum(school_data_complete['budget'].unique())
avg_math = school_data_complete['math_score'].mean()
avg_reading = school_data_complete['reading_score'].mean()
# Find pass rates using loc
math_pass = (len(school_data_complete.loc[school_data_complete['math_score']>= 70, :])/num_students)*100
read_pass = (len(school_data_complete.loc[school_data_complete['reading_score']>=70, :])/num_students)*100
pass_rate = (math_pass + read_pass)/2
# Create summary dataframe
district_summary = [{'Number of Schools':num_schools,
                   'Number of Students':num_students,
                   'Total Budget':tot_budget,
                   'Average Math Score':avg_math,
                   'Average Reading Score':avg_reading,
                   'Overall Pass Rate':pass_rate,
                   'Math Pass Rate':math_pass,
                   'Reading Pass Rate':read_pass}]
district_summary_df = pd.DataFrame(district_summary)
# Clean up formatting for numbers in summary df
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map("${:,}".format)
district_summary_df['Number of Students'] = district_summary_df['Number of Students'].map('{:,}'.format)
district_summary_df['Math Pass Rate']=district_summary_df['Math Pass Rate'].map('{:.1f}%'.format)
district_summary_df['Reading Pass Rate']=district_summary_df['Reading Pass Rate'].map('{:.1f}%'.format)
district_summary_df['Average Math Score']=district_summary_df['Average Math Score'].map('{:.1f}'.format)
district_summary_df['Average Reading Score']=district_summary_df['Average Reading Score'].map('{:.1f}'.format)
district_summary_df['Overall Pass Rate']=district_summary_df['Overall Pass Rate'].map('{:.1f}%'.format)
# Reorder the summary df so it makes more sense
reform_d_summ = district_summary_df[['Number of Schools','Number of Students','Average Math Score','Average Reading Score',
                                    'Math Pass Rate','Reading Pass Rate','Overall Pass Rate','Total Budget']]
reform_d_summ

,Number of Schools,Number of Students,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate,Total Budget
0,15,"39,170",79.0,81.9,75.0%,85.8%,80.4%,"$24,649,428"


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [17]:
# Take grouped average of data
summ_df = grouped_by_school.mean()
# Delete unnecessary columns
del summ_df['Student ID']
del summ_df['School ID']
# Rename summary table columns
summ_df = summ_df.rename(columns={'reading_score':'Average Reading Score','math_score':'Average Math Score',
                                  'size':'Total Students','budget':'Total School Budget','school_name':'School Name'})
# Count the number of students grouped by school
stu_ct = grouped_by_school.count()
# Count the number of students who passed grouped by school
math_pass_df = school_data_complete.loc[school_data_complete['math_score']>= 70, :]
read_pass_df = school_data_complete.loc[school_data_complete['reading_score']>=70, :]
num_pass_math = math_pass_df.groupby('school_name').count()
num_pass_read = read_pass_df.groupby('school_name').count()
# Calculate percentages and add to summary
per_math_pass = (num_pass_math['math_score']/stu_ct['math_score'])*100
per_read_pass = (num_pass_read['reading_score']/stu_ct['reading_score'])*100
summ_df['Math Pass Rate'] = per_math_pass
summ_df['Reading Pass Rate'] = per_read_pass
# Calculate overall pass rates and add to summary
overall_pass = (summ_df['Math Pass Rate']+ summ_df['Reading Pass Rate'])/2
summ_df['Overall Pass Rate'] = overall_pass
# Reset index
summ_df.reset_index(inplace=True)
summ_df = summ_df.rename(columns={'school_name':'School Name'})
# Save everything before its formatted
summ_df_unf = summ_df

In [18]:
# Make it pretty
summ_df['Average Reading Score'] = summ_df['Average Reading Score'].map('{:.1f}'.format)
summ_df['Average Math Score'] = summ_df['Average Math Score'].map('{:.1f}'.format)
summ_df['Total Students'] = summ_df['Total Students'].map('{:,.0f}'.format)
summ_df['Total School Budget'] = summ_df['Total School Budget'].map('${:,.0f}'.format)
summ_df['Math Pass Rate'] = summ_df['Math Pass Rate'].map('{:.1f}%'.format)
summ_df['Reading Pass Rate'] = summ_df['Reading Pass Rate'].map('{:.1f}%'.format)
summ_df['Overall Pass Rate'] = summ_df['Overall Pass Rate'].map('{:.1f}%'.format)
summ_df

,School Name,Average Reading Score,Average Math Score,Total Students,Total School Budget,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
0,Bailey High School,81.0,77.0,"4,976","$3,124,928",66.7%,81.9%,74.3%
1,Cabrera High School,84.0,83.1,"1,858","$1,081,356",94.1%,97.0%,95.6%
2,Figueroa High School,81.2,76.7,"2,949","$1,884,411",66.0%,80.7%,73.4%
3,Ford High School,80.7,77.1,"2,739","$1,763,916",68.3%,79.3%,73.8%
4,Griffin High School,83.8,83.4,"1,468","$917,500",93.4%,97.1%,95.3%
5,Hernandez High School,80.9,77.3,"4,635","$3,022,020",66.8%,80.9%,73.8%
6,Holden High School,83.8,83.8,427,"$248,087",92.5%,96.3%,94.4%
7,Huang High School,81.2,76.6,"2,917","$1,910,635",65.7%,81.3%,73.5%
8,Johnson High School,81.0,77.1,"4,761","$3,094,650",66.1%,81.2%,73.6%
9,Pena High School,84.0,83.8,962,"$585,858",94.6%,95.9%,95.3%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [19]:
# Take top 5 for overall passing rate
summ_top = summ_df_unf.sort_values('Overall Pass Rate', ascending = False)
summ_top = summ_top.reset_index(drop=True)
summ_top = summ_top.iloc[0:5,:]
summ_top

,School Name,Average Reading Score,Average Math Score,Total Students,Total School Budget,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
0,Cabrera High School,84.0,83.1,"1,858","$1,081,356",94.1%,97.0%,95.6%
1,Griffin High School,83.8,83.4,"1,468","$917,500",93.4%,97.1%,95.3%
2,Pena High School,84.0,83.8,962,"$585,858",94.6%,95.9%,95.3%
3,Thomas High School,83.8,83.4,"1,635","$1,043,130",93.3%,97.3%,95.3%
4,Wilson High School,84.0,83.3,"2,283","$1,319,574",93.9%,96.5%,95.2%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [20]:
# Take lowest 5
summ_bottom = summ_df_unf.sort_values('Overall Pass Rate', ascending = True)
summ_bottom = summ_bottom.reset_index(drop=True)
summ_bottom = summ_bottom.iloc[0:5,:]
summ_bottom

,School Name,Average Reading Score,Average Math Score,Total Students,Total School Budget,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
0,Rodriguez High School,80.7,76.8,"3,999","$2,547,363",66.4%,80.2%,73.3%
1,Figueroa High School,81.2,76.7,"2,949","$1,884,411",66.0%,80.7%,73.4%
2,Huang High School,81.2,76.6,"2,917","$1,910,635",65.7%,81.3%,73.5%
3,Johnson High School,81.0,77.1,"4,761","$3,094,650",66.1%,81.2%,73.6%
4,Ford High School,80.7,77.1,"2,739","$1,763,916",68.3%,79.3%,73.8%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [21]:
# Find each grade and save them as separate dfs
stu_9 = school_data_complete.loc[school_data_complete['grade'] == '9th']
stu_10 = school_data_complete.loc[school_data_complete['grade'] == '10th']
stu_11 = school_data_complete.loc[school_data_complete['grade'] == '11th']
stu_12 = school_data_complete.loc[school_data_complete['grade'] == '12th']
# Take mean of each grouped by school
grp_stu9 = stu_9.groupby('school_name').mean()
grp_stu10 = stu_10.groupby('school_name').mean()
grp_stu11 = stu_11.groupby('school_name').mean()
grp_stu12 = stu_12.groupby('school_name').mean()
# Isolate math score
math_9 = grp_stu9['math_score']
math_10 = grp_stu10['math_score']
math_11 = grp_stu11['math_score']
math_12 = grp_stu12['math_score']
# Create summary dataframe
math_summ = pd.DataFrame({'9th Grade':math_9,
                         '10th Grade':math_10,
                         '11th Grade':math_11,
                         '12th Grade':math_12})
# Make it pretty
math_summ['9th Grade'] = math_summ['9th Grade'].map('{:.1f}'.format)
math_summ['10th Grade'] = math_summ['10th Grade'].map('{:.1f}'.format)
math_summ['11th Grade'] = math_summ['11th Grade'].map('{:.1f}'.format)
math_summ['12th Grade'] = math_summ['12th Grade'].map('{:.1f}'.format)
math_summ.reset_index(inplace=True)
math_summ = math_summ.rename(columns={'school_name':'School Name'})
math_summ

,School Name,9th Grade,10th Grade,11th Grade,12th Grade
0,Bailey High School,77.1,77.0,77.5,76.5
1,Cabrera High School,83.1,83.2,82.8,83.3
2,Figueroa High School,76.4,76.5,76.9,77.2
3,Ford High School,77.4,77.7,76.9,76.2
4,Griffin High School,82.0,84.2,83.8,83.4
5,Hernandez High School,77.4,77.3,77.1,77.2
6,Holden High School,83.8,83.4,85.0,82.9
7,Huang High School,77.0,75.9,76.4,77.2
8,Johnson High School,77.2,76.7,77.5,76.9
9,Pena High School,83.6,83.4,84.3,84.1


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [22]:
# Access average reading scores per grade grouped by school
read_9 = grp_stu9['reading_score']
read_10 = grp_stu10['reading_score']
read_11 = grp_stu11['reading_score']
read_12 = grp_stu12['reading_score']
# New dataframe
read_summ = pd.DataFrame({'9th Grade':read_9,
                         '10th Grade':read_10,
                         '11th Grade':read_11,
                         '12th Grade':read_12})
# Format the data
read_summ['9th Grade'] = read_summ['9th Grade'].map('{:.1f}'.format)
read_summ['10th Grade'] = read_summ['10th Grade'].map('{:.1f}'.format)
read_summ['11th Grade'] = read_summ['11th Grade'].map('{:.1f}'.format)
read_summ['12th Grade'] = read_summ['12th Grade'].map('{:.1f}'.format)
read_summ.reset_index(inplace=True)
read_summ = read_summ.rename(columns={'school_name':'School Name'})
read_summ

,School Name,9th Grade,10th Grade,11th Grade,12th Grade
0,Bailey High School,81.3,80.9,80.9,80.9
1,Cabrera High School,83.7,84.3,83.8,84.3
2,Figueroa High School,81.2,81.4,80.6,81.4
3,Ford High School,80.6,81.3,80.4,80.7
4,Griffin High School,83.4,83.7,84.3,84.0
5,Hernandez High School,80.9,80.7,81.4,80.9
6,Holden High School,83.7,83.3,83.8,84.7
7,Huang High School,81.3,81.5,81.4,80.3
8,Johnson High School,81.3,80.8,80.6,81.2
9,Pena High School,83.8,83.6,84.3,84.6


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [23]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [24]:
#school_data_complete - full dataframe
#need to first find spending per student for each school
school_df = grouped_by_school.mean()
school_df.reset_index(inplace=True)
school_df['spending_per_student'] = school_df['budget']/school_df['size']
del school_df['School ID']
del school_df['Student ID']
del school_df['reading_score']
del school_df['math_score']
# Merge the data to fill in the spending per student
school_merge = pd.merge(school_data_complete,school_df,on='school_name',how='left')
del school_merge['Student ID']
del school_merge['School ID']
del school_merge['gender']
del school_merge['grade']
school_merge
# Bin the data based on spending per student
school_merge["spending_group"] = pd.cut(school_merge["spending_per_student"], spending_bins, labels=group_names)
# Group by spending group
school_spend = school_merge.groupby('spending_group')
# Find other data listed
summ_spend = school_spend.mean()
count_spend = school_spend['student_name'].count()
# Find number of kids passing within each group --> percentage
math_pass = school_merge.loc[school_data_complete['math_score']>= 70, :]
read_pass = school_merge.loc[school_data_complete['reading_score']>=70, :]
math_spend_num = math_pass.groupby('spending_group').count()
read_spend_num = read_pass.groupby('spending_group').count()
summ_spend['Math Pass Rate'] = (math_spend_num['student_name']/count_spend)*100
summ_spend['Reading Pass Rate'] = (read_spend_num['student_name']/count_spend)*100
# Take out some unneccessary columns
del summ_spend['size_x']
del summ_spend['size_y']
del summ_spend['budget_x']
del summ_spend['budget_y']
del summ_spend['spending_per_student']
# Format the summary table
summ_spend = summ_spend.rename(columns={'math_score':'Average Math Score','reading_score':'Average Reading Score'})
summ_spend.reset_index(inplace=True)
summ_spend = summ_spend.rename(columns={'spending_group':'Spending Group'})
summ_spend['Average Math Score'] = summ_spend['Average Math Score'].map('{:.1f}'.format)
summ_spend['Average Reading Score'] = summ_spend['Average Reading Score'].map('{:.1f}'.format)
summ_spend['Math Pass Rate'] = summ_spend['Math Pass Rate'].map('{:.1f}%'.format)
summ_spend['Reading Pass Rate'] = summ_spend['Reading Pass Rate'].map('{:.1f}%'.format)
summ_spend

,Spending Group,Average Reading Score,Average Math Score,Math Pass Rate,Reading Pass Rate
0,<$585,84.0,83.4,93.7%,96.7%
1,$585-615,83.8,83.5,94.1%,95.9%
2,$615-645,81.4,78.1,71.4%,83.6%
3,$645-675,81.0,77.0,66.2%,81.1%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [25]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [28]:
# Bin the data based on the size of school
school_data_complete["size_group"] = pd.cut(school_data_complete["size"], size_bins, labels=group_names)
# Group by the size
grpby_size = school_data_complete.groupby('size_group')
summ_size = grpby_size.mean()
bysize_count = grpby_size.count()
# Find the rest of the data for the summary table
math_pass_df = school_data_complete.loc[school_data_complete['math_score']>= 70, :]
read_pass_df = school_data_complete.loc[school_data_complete['reading_score']>=70, :]
pass_math_size = math_pass_df.groupby('size_group').count()
pass_read_size = read_pass_df.groupby('size_group').count()
summ_size['Math Pass Rate'] = (pass_math_size['student_name']/bysize_count['student_name'])*100
summ_size['Reading Pass Rate'] = (pass_read_size['student_name']/bysize_count['student_name'])*100

#format the summary table
del summ_size['Student ID']
del summ_size['School ID']
del summ_size['budget']
del summ_size['size']
summ_size.reset_index(inplace=True)
summ_size['reading_score'] = summ_size['reading_score'].map('{:.1f}'.format)
summ_size['math_score'] = summ_size['math_score'].map('{:.1f}'.format)
summ_size['Math Pass Rate'] = summ_size['Math Pass Rate'].map('{:.1f}%'.format)
summ_size['Reading Pass Rate'] = summ_size['Reading Pass Rate'].map('{:.1f}%'.format)
summ_size = summ_size.rename(columns={'size_group':'Size Group','reading_score':'Average Reading Score','math_score':'Average Math Score'})
summ_size

,Size Group,Average Reading Score,Average Math Score,Math Pass Rate,Reading Pass Rate
0,Small (<1000),84.0,83.8,94.0%,96.0%
1,Medium (1000-2000),83.9,83.4,93.6%,96.8%
2,Large (2000-5000),81.2,77.5,68.7%,82.1%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [49]:
# Group overall data by school type
grpby_type = school_data_complete.groupby('type')
summ_type = grpby_type.mean()
count_type = grpby_type.count()
# Find other data for summary table
pass_math_type = math_pass_df.groupby('type').count()
pass_read_type = read_pass_df.groupby('type').count()
summ_type['Math Pass Rate'] = (pass_math_type['Student ID']/count_type['Student ID'])*100
summ_type['Reading Pass Rate'] = (pass_read_type['Student ID']/count_type['Student ID'])*100
# Format summary table
del summ_type['Student ID']
del summ_type['School ID']
del summ_type['size']
del summ_type['budget']
summ_type.reset_index(inplace=True)
summ_type = summ_type.rename(columns={'type':'Type','reading_score':'Average Reading Score','math_score':'Average Math Score'})
summ_type['Average Reading Score'] = summ_type['Average Reading Score'].map('{:.1f}'.format)
summ_type['Average Math Score'] = summ_type['Average Math Score'].map('{:.1f}'.format)
summ_type['Math Pass Rate'] = summ_type['Math Pass Rate'].map('{:.1f}%'.format)
summ_type['Reading Pass Rate'] = summ_type['Reading Pass Rate'].map('{:.1f}%'.format)
summ_type

,Type,Average Reading Score,Average Math Score,Math Pass Rate,Reading Pass Rate
0,Charter,83.9,83.4,93.7%,96.6%
1,District,81.0,77.0,66.5%,80.9%
